# Setup en colab

Crea un shortcut en tu drive de los [datos](https://drive.google.com/drive/folders/1djjceNkO42vrB10PubYTzQydfccPbzdB?usp=sharing)



In [1]:
# necesario si vas a usar los embeddings de titulo
!wget -O title_fasttext.bin https://drive.google.com/file/d/1eB9X-9XmDMP1Hv_892JKLCpk8JePzhG3/view?usp=sharing
!pip install fasttext 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.0 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3163642 sha256=a77018376da3a6ff6a8331dfaea5c95ad568c3a7bc06da71f2c035bfb3a6a0c2
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [2]:
# Clonamos el repo para usar el codigo de la lib
!git clone https://github.com/elsonidoq/ml-practico-2022.git
!cd ml-practico-2022; git pull

fatal: destination path 'ml-practico-2022' already exists and is not an empty directory.


In [4]:
import sys
sys.path.append('ml-practico-2022/lib')

In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Entrenamos un modelo

In [6]:
from taller_model_selection.evaluate import load_train_dev_test

(X_train, y_train), (X_dev, y_dev), test = load_train_dev_test('/content/gdrive/MyDrive/taller-model-selection-data')

{'pct(train)': 0.809998757918271, 'pct(dev)': 0.09000124208172898, 'pct(test)': 0.1}


In [7]:
(
    max([e['created_on'] for e in X_train]), 
    max([e['created_on'] for e in X_dev]), 
    max([e['created_on'] for e in test[0]])
)

('2021-05-01', '2021-05-01', '2021-05-01')

In [8]:
import numpy as np

mean_price = np.mean(y_train)
median_price = np.median(y_train)

In [9]:
from taller_model_selection.evaluate import Evaluator

evaluator = Evaluator(X_train, y_train, X_dev, y_dev)

In [10]:
evaluator.eval_prediction('mean_price', y_hat_train=[mean_price] * len(y_train), y_hat_dev=[mean_price] * len(y_dev))

{'name': 'mean_price', 'train': 448215.69625457143, 'dev': 457302.60238727595}

In [11]:
evaluator.eval_prediction('median_price', y_hat_train=[median_price] * len(y_train), y_hat_dev=[median_price] * len(y_dev))

{'name': 'median_price',
 'train': 463830.09217220347,
 'dev': 472424.57116989064}

In [12]:
import pandas as pd

pd.DataFrame(evaluator.evaluations)

,name,train,dev
0,mean_price,448215.696255,457302.602387
1,median_price,463830.092172,472424.571170


# Fit the model

In [13]:
from taller_model_selection.transformers import FeatureProjection, TargetEncoder, ZonePriceM2, PretrainedFastTextTransformer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

features_pipe =  make_union(
    PretrainedFastTextTransformer('title_fasttext.bin', 'title'),

    make_pipeline(
        FeatureProjection(['rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered']),
        SimpleImputer()
    ),
    # make_pipeline(
    #     FeatureProjection(['l3']), 
    #     SimpleImputer(strategy='most_frequent'),
    #     OneHotEncoder(sparse=False, drop='first')
    # ), 
    # Podes probar usar target encoder, one hot, o ambos
    TargetEncoder('l3'),
    ZonePriceM2(n_neighbours=30),
    make_pipeline(
        FeatureProjection(['l4']), 
        SimpleImputer(strategy='constant'),
        OneHotEncoder(sparse=False, drop='first')
    ), 
)

lr_pipe = make_pipeline(
    features_pipe,
    LinearRegression()
)

rf_pipe = make_pipeline(
    features_pipe,
    RandomForestRegressor(n_estimators=20)
)

In [14]:
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('pretrainedfasttexttransformer',
                                                 PretrainedFastTextTransformer(field='title',
                                                                               fname='title_fasttext.bin')),
                                                ('pipeline-1',
                                                 Pipeline(steps=[('featureprojection',
                                                                  FeatureProjection(fields=['rooms',
                                                                                            'bedrooms',
                                                                                            'bathrooms',
                                                                                            'surface_total',
                                                                                            'surface_covered'])),

In [15]:
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('pretrainedfasttexttransformer',
                                                 PretrainedFastTextTransformer(field='title',
                                                                               fname='title_fasttext.bin')),
                                                ('pipeline-1',
                                                 Pipeline(steps=[('featureprojection',
                                                                  FeatureProjection(fields=['rooms',
                                                                                            'bedrooms',
                                                                                            'bathrooms',
                                                                                            'surface_total',
                                                                                            'surface_covered'])),

In [16]:
evaluator.eval_pipe('lr', lr_pipe)

{'name': 'lr', 'train': 352315.0487310766, 'dev': 359538.5058582858}

In [17]:
evaluator.eval_pipe('rf', rf_pipe)

{'name': 'rf', 'train': 89276.66817025465, 'dev': 249825.53522332892}

In [18]:
df = pd.DataFrame(evaluator.evaluations)
df

,name,train,dev
0,mean_price,448215.696255,457302.602387
1,median_price,463830.092172,472424.571170
2,lr,352315.048731,359538.505858
3,rf,89276.668170,249825.535223


In [19]:
df['pct_lift'] = df.dev / df.dev.max()
df.sort_values('dev')

,name,train,dev,pct_lift
3,rf,89276.668170,249825.535223,0.528816
2,lr,352315.048731,359538.505858,0.761050
0,mean_price,448215.696255,457302.602387,0.967991
1,median_price,463830.092172,472424.571170,1.000000


In [20]:
X_test, y_test = test

In [21]:
from taller_model_selection.evaluate import rmse

rmse(y_test, rf_pipe.predict(X_test))


240129.78926577826